## Calibración de película y aplicación a un archivo tiff 
Dosepy versión 0.6.0

### Importar librerias

In [1]:
from Dosepy.tools.image import load
from pathlib import Path

ModuleNotFoundError: No module named 'calibration'

### Definir las dosis usadas para calibración

In [ ]:
imparted_doses = [0, 0.5, 1, 1.5, 2, 3, 5, 8, 10]

### Lectura de archivo

Archivo tiff con películas irradiadas a los valores de dosis previos.

In [ ]:
#file_path = Path("/home/luis/Downloads") / "POS48_1.tif"  # Office PC
file_path = Path("/home/luis/Descargas") / "POS48_1.tif" # Personal laptop
cal_image = load(file_path, for_calib = True)

### Generar curva de calibración
Se utilizará el canal rojo, una ROI rectangular de 16 mm x 8 mm y un ajuste a una función racional.

In [ ]:
cal = cal_image.get_calibration(doses = imparted_doses, channel = "R", roi = (16, 8), func = "RF")
cal.plot(color = 'red')

### Mostrar ROIs y valores medios de pixel.

In [ ]:
cal_image.get_stat(ch = 'B', roi = (16, 8), show = True)

## Aplicar curva a un nuevo archivo tiff


### Lectura de archivo

In [ ]:
#verif_path = Path("/home/luis/Downloads") / "VERIFICACION_1.tif"  # Office PC
verif_path = Path("/home/luis/Descargas") / "VERIFICACION_1.tif" # Personal laptop
verif = load(verif_path)

verif.plot()

### Aplicar curva 

In [ ]:
dose_img = verif.to_dose(cal)

### Mostrar distribución

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
fig, ax = plt.subplots(ncols=1)

# plot and save the
# color "mappable" object returned by ax.imshow
max_dose = np.percentile(dose_img.array, [99.9])[0]
#pos = ax.imshow(dose_img.array, cmap='turbo')
pos = ax.imshow(dose_img.array, cmap='nipy_spectral')
#pos = ax.imshow(dose_img.array)
pos.set_clim(-.05, max_dose)
#pos.set_clim(0, max_dose)

# add the colorbar using the figure's method,
# telling which mappable we're talking about and
# which axes object it should be near
fig.colorbar(pos, ax=ax)


## Guardar como archivo tiff (de utilidad para análisis con ImageJ)

### Numpy to bytes and cGy

In [ ]:
import imageio.v3 as iio
#%matplotlib qt

data = np.int64(dose_img.array*100) # Gy to cGy

np_tif = data.astype(np.uint16)
png_encoded = iio.imwrite("<bytes>", np_tif, extension=".tif")


### Guardar como tiff

In [ ]:
with open('test.tif', 'wb') as f: 
    f.write(png_encoded)

### Mostrar dosis promedio en ROI (...en desarrollo)

In [ ]:
import numpy as np
from tools.calibration import polynomial_g3, rational_func

mean_pixel, _ = verif.get_stat(ch = "R", roi = (30, 10), show = True)
mean_pixel = sorted(mean_pixel, reverse = True)

# Pixel normalization 
norm_pixel = np.array(mean_pixel)/mean_pixel[0]
dose_from_rational = rational_func(norm_pixel, *cal.popt)
print(f"Dosis por ajuste racional: {dose_from_rational}")


In [ ]:
doses_in_central_rois = verif.doses_in_central_rois(cal, roi = (20, 8), show=True)
print(doses_in_central_rois)

In [ ]:
type(doses_in_central_rois)

In [ ]:
dose_img.save_as_tif("tif_from_function")

In [ ]:
%matplotlib qt

In [ ]:
plt.imshow(dose_img.array)

## Calibración con vidrio y tapa

In [ ]:
#file_path_3 = Path("/home/luis/Downloads") / "POS48_3.tif"  # Office PC
file_path_3 = Path("/home/luis/Descargas") / "POS48_3.tif" # Personal laptop
cal_image_3 = load(file_path_3, for_calib = True)

In [ ]:
cal_3 = cal_image_3.get_calibration(doses = imparted_doses, channel = "B", roi = (16, 8), func = "P3")
cal_3.plot(color = 'blue')

In [ ]:
cal_image_3.get_stat(ch = 'B', roi = (16, 8), show = True)

In [ ]:
#verif_path_3 = Path("/home/luis/Downloads") / "VERIFICACION_3.tif"  # Office PC
verif_path_3 = Path("/home/luis/Descargas") / "VERIFICACION_3.tif" # Personal laptop
verif_3 = load(verif_path_3)

In [ ]:
mean_pixel_3, _ = verif_3.get_stat(ch = "B", roi = (30, 10), show = True)
mean_pixel_3 = sorted(mean_pixel_3, reverse = True)
#mean_pixel

optical_density_3 = -np.log10(np.array(mean_pixel_3)/mean_pixel_3[0])

dosis_from_fit_3 = polynomial_g3(optical_density_3, *cal_3.popt)
dosis_from_fit_3[dosis_from_fit_3 < 0] = 0 # Remove doses < 0

print(f"Dosis por ajuste: {dosis_from_fit_3}")
